In [1]:
import os, sys
import numpy as np
import pandas as pd
import joblib

In [2]:
working_dir = "/home/ilya/Box/GPCRapa/github/GPCRapa/"


dir_path = os.path.join(working_dir,"test")# path to working directory
out_path = os.path.join(working_dir,"test")# path to output directory

In [10]:
path_to_features = os.path.join(working_dir,"Files/resources/calculated_features1_03_21.csv")
path_to_state_mapping = os.path.join(working_dir,"Files/resources/data_GPCR.csv")
path_to_mod_desc = os.path.join(working_dir,"Files/resources/models_description.csv")

In [11]:
pdb_list = [
    '6C1R','4NTJ','5NJ6','5VBL','4DKL','4DJH','2YDO','5UIW','6KO5','5ZKQ',
    '5G53','5XSZ','5XJM','5GLH','5UNH','2YDV','5UEN','3V2Y','4UHR','5T1A',
    '6DO1','6IIV','4EIY','4ZUD','5WB1','6ME2','4XT1','5XPR','6LFL','5ZBQ',
    '6GDG','4PXZ','6M9T','6D26','6LFO','5DHH','4XNW','6B73','3VW7','4XT3',
    '5TZY','6D9H','6DDE','4EJ4','6ME8','4Z36','4RWD']

# Import data

In [12]:
feat = pd.read_csv(path_to_features)
mod_descr = pd.read_csv(path_to_mod_desc)
state_mapping = pd.read_csv(path_to_state_mapping)


#make a dict with conformational state annotation for the dataset structures
state_mapping_cut = state_mapping[state_mapping["PDB code"].isin(pdb_list)][["PDB code", "State"]]
state_mapping_dict = {}
for item1, item2 in zip(state_mapping_cut["PDB code"].tolist(), state_mapping_cut["State"].tolist()):
    state_mapping_dict[item1] = item2
active_structures = state_mapping_cut[state_mapping_cut["State"] == "active"]
active_structures = active_structures["PDB code"].tolist()
inactive_structures = state_mapping_cut[state_mapping_cut["State"] == "inactive"]
inactive_structures = inactive_structures["PDB code"].tolist()

In [13]:
mod_descr

,Unnamed: 0,Model,Training_set_actve,Training_set_full,Test_set_active,Test_set_full,pres,rec,acc,balans_acc
0,0,model_0,"['4XT3', '5G53', '6B73', '6D9H', '6DDE', '6DO1...","['4XT3', '5G53', '6B73', '6D9H', '6DDE', '6DO1...","['4XT1', '5XJM']","['4XT1', '5XJM', '3V2Y', '4DKL', '4XNW', '5T1A...",1.0,1.0,1.0,1.0
1,1,model_1,"['4XT3', '5G53', '5XJM', '6B73', '6DDE', '6DO1...","['4XT3', '5G53', '5XJM', '6B73', '6DDE', '6DO1...","['4XT1', '6D9H']","['4XT1', '6D9H', '4EJ4', '5DHH', '5VBL', '5XPR...",1.0,1.0,1.0,1.0
2,2,model_2,"['4XT3', '5G53', '5XJM', '6B73', '6D9H', '6DO1...","['4XT3', '5G53', '5XJM', '6B73', '6D9H', '6DO1...","['4XT1', '6DDE']","['4XT1', '6DDE', '4EJ4', '5NJ6', '5T1A', '5XPR...",1.0,1.0,1.0,1.0
3,3,model_3,"['4XT1', '5XJM', '6B73', '6D9H', '6DDE', '6DO1...","['4XT1', '5XJM', '6B73', '6D9H', '6DDE', '6DO1...","['4XT3', '5G53']","['4XT3', '5G53', '4EJ4', '4XNW', '4Z36', '5VBL...",1.0,1.0,1.0,1.0
4,4,model_4,"['4XT1', '5G53', '5XJM', '6B73', '6D9H', '6DDE...","['4XT1', '5G53', '5XJM', '6B73', '6D9H', '6DDE...","['4XT3', '6DO1']","['4XT3', '6DO1', '4XNW', '5XPR', '5ZBQ', '6D26...",1.0,1.0,1.0,1.0
5,5,model_5,"['4XT1', '5G53', '5XJM', '6B73', '6D9H', '6DDE...","['4XT1', '5G53', '5XJM', '6B73', '6D9H', '6DDE...","['4XT3', '6LFO']","['4XT3', '6LFO', '4DKL', '5NJ6', '5ZBQ', '6D26...",1.0,1.0,1.0,1.0
6,6,model_6,"['4XT1', '4XT3', '5XJM', '6B73', '6DDE', '6DO1...","['4XT1', '4XT3', '5XJM', '6B73', '6DDE', '6DO1...","['5G53', '6D9H']","['5G53', '6D9H', '4DJH', '4EJ4', '5XPR', '5ZKQ...",1.0,1.0,1.0,1.0
7,7,model_7,"['4XT1', '4XT3', '5G53', '6B73', '6DDE', '6DO1...","['4XT1', '4XT3', '5G53', '6B73', '6DDE', '6DO1...","['5XJM', '6D9H']","['5XJM', '6D9H', '4DKL', '4XNW', '4ZUD', '5NJ6...",1.0,1.0,1.0,1.0
8,8,model_8,"['4XT1', '4XT3', '5G53', '6B73', '6D9H', '6DDE...","['4XT1', '4XT3', '5G53', '6B73', '6D9H', '6DDE...","['5XJM', '6LFO']","['5XJM', '6LFO', '4EJ4', '4NTJ', '4XNW', '5VBL...",1.0,1.0,1.0,1.0
9,9,model_9,"['4XT1', '4XT3', '5G53', '5XJM', '6B73', '6DDE...","['4XT1', '4XT3', '5G53', '5XJM', '6B73', '6DDE...","['6D9H', '6DO1']","['6D9H', '6DO1', '3VW7', '4DKL', '5NJ6', '5XPR...",1.0,1.0,1.0,1.0


In [17]:
# in case of reproducing alredy trained RF models
train_active_set = [f.rstrip("'").lstrip("'") for f in mod_descr[mod_descr["Model"] == "model_19"].iloc[0, 2].rstrip(']"').lstrip('"[').split(", ")]
train_inactive_set = [f for f in [f.rstrip("'").lstrip("'") for f in mod_descr[mod_descr["Model"] == "model_19"].iloc[0, 3].rstrip(']"').lstrip('"[').split(", ")] if f not in train_active_set]

# Splitting

In [14]:
def custom_split(data:pd.DataFrame(),
                 active_pdb_list:list,
                 inactive_pdb_list:list,
                 path_to_state_mapping:str,
                 a_train = 8, ina_train =22) -> np.array:
    
    """Creates custom split for the GPCR class A MD trajectories dataset, with the respect
    of the dataset initial proportion. All frames from the trajectory go either in training
    or test set. 

    Args:
    
      data:
          Dataframe with calculated features of each MD frame of the dataset.
          
      active_pdb_list:
          List which contains pdb ids that represent active structures from the dataset.
          
      inactive_pdb_list:
          List which contains pdb ids that represent inactive structures from the dataset.
          
      path_to_state_mapping:
          Path to the Dataframe that contains mapping for all the dataset structures,
          particularly their state classification.
          
      a_train:
          Number of active structures in training set (Default - 8).
          
      ina_train:
          Number of active structures in training set (Default - 22).

    Returns:
    
      Custom split of the dataset to training and test set with labelling. 
    """
    
    import random
    #split
    a_pdb_train = random.sample(active_pdb_list, a_train)
    a_pdb_test = [f for f in active_pdb_list if f not in a_pdb_train]
    ina_pdb_train = random.sample(inactive_pdb_list, ina_train)
    ina_pdb_test = [f for f in inactive_pdb_list if f not in ina_pdb_train]
    
    
    #making of the mapping dict
    state_mapping = pd.read_csv(path_to_state_mapping)
    state_mapping_cut = state_mapping[state_mapping["PDB code"].isin(pdb_list)][["PDB code", "State"]]
    state_mapping_dict = {}
    for item1, item2 in zip(state_mapping_cut["PDB code"].tolist(), state_mapping_cut["State"].tolist()):
        state_mapping_dict[item1] = item2
    
    
    #data processing 
    data["Pdb_names"] = [f.split("_")[0] for f in feat["Unnamed: 0"]]
    cl = [f for f in feat.columns.tolist() if f not in ["Unnamed: 0", "Pdb_names"]]
    
    
    #training set preparation
    train_df = data[data["Pdb_names"].isin(a_pdb_train+ina_pdb_train)].copy()
    st =[]
    for item in train_df["Unnamed: 0"].tolist():
        if state_mapping_dict[item.split("_")[0]] == "active":
            st.append(1)
        elif state_mapping_dict[item.split("_")[0]] == "inactive":
            st.append(0)
    train_labels = np.array(st)
    train_array = train_df[cl].to_numpy()
    
    
    #test set preparation
    test_df = data[data["Pdb_names"].isin(a_pdb_test+ina_pdb_test)].copy()
    print("Test_set_structures:", a_pdb_test+ina_pdb_test)
    sf =[]
    for item in test_df["Unnamed: 0"].tolist():
        if state_mapping_dict[item.split("_")[0]] == "active":
            sf.append(1)
        elif state_mapping_dict[item.split("_")[0]] == "inactive":
            sf.append(0)
    test_labels = np.array(sf)
    test_array = test_df[cl].to_numpy()
    
    
    
    return train_array, test_array, train_labels, test_labels

In [18]:
X_train, X_test, y_train, y_test = custom_split(feat, active_structures, inactive_structures, path_to_state_mapping)

Test_set_structures: ['5XJM', '6LFO', '3V2Y', '4DJH', '4RWD', '5ZBQ', '6C1R', '6ME2']


In [18]:
#model_19 reproduction

In [19]:
X_train = feat[feat["Pdb_names"].isin(train_active_set + train_inactive_set)].copy().drop(columns=["Unnamed: 0", "Pdb_names"], axis = 1).to_numpy()
y_train = [1 if state_mapping_dict[f.split("_")[0]] == "active" else 0 for f in feat[feat["Pdb_names"].isin(train_active_set + train_inactive_set)]["Pdb_names"].to_list()]
X_train

array([[ 4.53303334,  5.09480943,  8.90566163, ..., 11.75284715,
         5.91643599,  8.04526827],
       [ 5.06099783,  6.18682764,  9.19739207, ..., 11.11570481,
         5.85888491,  7.42189561],
       [ 5.44398728,  5.19628306,  8.98876807, ...,  9.96657102,
         5.52738764,  5.87586497],
       ...,
       [ 4.4115219 ,  5.52447273,  7.5842271 , ...,  6.09866479,
         6.59354574,  5.88375006],
       [ 5.02061413,  5.58184523,  8.24997294, ...,  6.25573668,
         6.42211093,  5.86233673],
       [ 4.49965099,  5.45993332,  8.1587748 , ...,  6.13005579,
         7.1526984 ,  6.98890419]])

# Training and analysis

# Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)

In [21]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
from sklearn.metrics import balanced_accuracy_score
print("Balanced_accuracy", balanced_accuracy_score(y_test, clf.predict(X_test)))
print("Accuracy", clf.score(X_test, y_test))

Balanced_accuracy 1.0
Accuracy 1.0


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1206
           1       1.00      1.00      1.00       402

    accuracy                           1.00      1608
   macro avg       1.00      1.00      1.00      1608
weighted avg       1.00      1.00      1.00      1608



# SVM

In [67]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm
clf = svm.SVC(kernel='rbf')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
clf.fit(X_train, y_train)

In [69]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1206
           1       1.00      1.00      1.00       402

    accuracy                           1.00      1608
   macro avg       1.00      1.00      1.00      1608
weighted avg       1.00      1.00      1.00      1608



# XGBoost

In [77]:
from xgboost import XGBClassifier
xg_clf = XGBClassifier(n_estimators=100)

xg_clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

[19:17:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1206
           1       1.00      1.00      1.00       402

    accuracy                           1.00      1608
   macro avg       1.00      1.00      1.00      1608
weighted avg       1.00      1.00      1.00      1608



/home/ilya/anaconda3/envs/modelling/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


# Cross-validation

Random splits without repeating

In [8]:
def custom_split_2_6_wo_repeats(data:pd.DataFrame(),
                                path_to_state_mapping:str,
                                active_pdb_list:list,
                                inactive_pdb_list:list,
                                a_train = 2,
                                n_in_str_max = 6,
                                n_in_str_min = 5,
                                n_samples_26 = 3,
                                n_samples_25 = 2,
                                seed = 42) -> list:
    
    
    """Creates custom split for the GPCR class A MD trajectories dataset for 5-fold CV, with the respect
    of the dataset initial proportion. All frames from the trajectory go either in training or test set. 

    Args:
    
      data:
          Dataframe with calculated features of each MD frame of the dataset.
     
      path_to_state_mapping:
      
          Path to the Dataframe that contains mapping for all the dataset structures, particularly their
          state classification.
          
      active_pdb_list:
          List which contains pdb ids that represent active structures from the dataset.
          
      inactive_pdb_list:
          List which contains pdb ids that represent inactive structures from the dataset.
          
      a_train:
          Number of active structures in training set (Default - 8).
          
      n_in_str_max:
          Maximum number of inactive strucutes in samples for CV (Default - 6).
          
      n_in_str_min:
          Minimum number of inactive strucutes in samples for CV (Default - 5).
          
      n_samples_26:
          Number of samples with 2 active and 6 inactive structures for CV (Default - 3).
          
      n_samples_25:
          Number of of samples with 2 active and 5 inactive structures for CV (Default - 2).
          
      seed:
          Random seed.
          
    Returns:
    
      List, containing the splits for 5-fold CV. Each custom split consists of training and
      test set with labelling.
    """
    
    
    import random
    #разбивка
    i=0
    cv_sets_list = []
    added_pdbs = []
    random.seed(seed)
    while i<n_samples_26:
        
        active_pdb_set = random.sample([f for f in active_pdb_list if f not in added_pdbs], a_train)
        inactive_pdb_set = random.sample([f for f in inactive_pdb_list if f not in added_pdbs], n_in_str_max)
        cv_sets_list.append(active_pdb_set+inactive_pdb_set)
        added_pdbs+=active_pdb_set+inactive_pdb_set
        i+=1
    
    i=0
    while i<n_samples_25:
        
        active_pdb_set = random.sample([f for f in active_pdb_list if f not in added_pdbs], a_train)
        inactive_pdb_set = random.sample([f for f in inactive_pdb_list if f not in added_pdbs], n_in_str_min)
        cv_sets_list.append(active_pdb_set+inactive_pdb_set)
        added_pdbs+=active_pdb_set+inactive_pdb_set
        i+=1
    
    print(cv_sets_list)
    print(len(list(set(added_pdbs))))
    
    #making of the mapping dict
    state_mapping = pd.read_csv(path_to_state_mapping)
    state_mapping_cut = state_mapping[state_mapping["PDB code"].isin(pdb_list)][["PDB code", "State"]]
    state_mapping_dict = {}
    for item1, item2 in zip(state_mapping_cut["PDB code"].tolist(), state_mapping_cut["State"].tolist()):
        state_mapping_dict[item1] = item2
    
    #data processing 
    data["Pdb_names"] = [f.split("_")[0] for f in feat["Unnamed: 0"]]
    cl = [f for f in feat.columns.tolist() if f not in ["Unnamed: 0", "Pdb_names"]]
    
    #CV sets preparation
    cv_sets=[]
    for test_set in cv_sets_list:
        train_sets=[f for f in cv_sets_list if f != test_set]
        
        
        #data for k-1 train
        train_dfs=[]
        for train_set in train_sets:
            train_df = data[data["Pdb_names"].isin(train_set)].copy()
            train_dfs.append(train_df)
        train_df = pd.concat(train_dfs)
        print(train_df.shape)
        
        
        #labels for k-1 train
        train_labels=[]
        for item in train_df["Unnamed: 0"].tolist():
            if state_mapping_dict[item.split("_")[0]] == "active":
                train_labels.append(1)
            elif state_mapping_dict[item.split("_")[0]] == "inactive":
                train_labels.append(0)
        
        train_labels = np.array(train_labels)
        train_array = train_df[cl].to_numpy()
        
        
        #data for test
        test_df = data[data["Pdb_names"].isin(test_set)].copy()
        print(test_df.shape)

        
        #labels for test
        test_labels=[]
        for item in test_df["Unnamed: 0"].tolist():
            if state_mapping_dict[item.split("_")[0]] == "active":
                test_labels.append(1)
            elif state_mapping_dict[item.split("_")[0]] == "inactive":
                test_labels.append(0)
        
        test_labels = np.array(test_labels)
        test_array = test_df[cl].to_numpy()
        
        
        cv_sets.append([[test_labels,test_array],[train_labels,train_array]])
        
        
    return cv_sets

In [9]:
cv_data_wo_rep_26 = custom_split_2_6_wo_repeats(feat, path_to_state_mapping,
                                                active_structures, inactive_structures,
                                                seed=42)

[['4XT3', '4XT1', '6IIV', '4XNW', '4RWD', '6ME8', '4EIY', '4DKL'], ['5XJM', '6DDE', '5XPR', '3VW7', '3V2Y', '4DJH', '4ZUD', '5DHH'], ['6GDG', '6LFO', '4EJ4', '5XSZ', '5NJ6', '6C1R', '5ZKQ', '6LFL'], ['6DO1', '5G53', '6D26', '5UIW', '4NTJ', '5ZBQ', '4Z36'], ['6D9H', '6B73', '5VBL', '5UEN', '5T1A', '6ME2', '6KO5']]
38
(6030, 40)
(1608, 40)
(6030, 40)
(1608, 40)
(6030, 40)
(1608, 40)
(6231, 40)
(1407, 40)
(6231, 40)
(1407, 40)


In [39]:
from sklearn.ensemble import RandomForestClassifier
cv_results = []
for set_ in cv_data_wo_rep_26:
    
    
    clf = RandomForestClassifier(n_estimators=100)
    
    
    test_set = set_[0]
    test_labels = test_set[0]
    test_array = test_set[1]
    
    
    train_set = set_[1]
    train_labels = train_set[0]
    train_array = train_set[1]
    
    clf.fit(train_array, train_labels)
    cv_results.append(clf.score(test_array, test_labels))
    
    
print(cv_results, np.mean(cv_results), np.std(cv_results))

[0.9968905472636815, 0.9975124378109452, 0.75, 1.0, 0.992181947405828] 0.9473169864960909 0.09869091047210349


In [43]:
joblib.dump(clf, '/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/rf_42.joblib') 

['/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/rf_42.joblib']

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm
cv_results = []
for set_ in cv_data_wo_rep_26:
    
    
    clf_svm = svm.SVC(kernel='rbf', probability=True)
    
    
    test_set = set_[0]
    test_labels = test_set[0]
    test_array = test_set[1]
    
    
    train_set = set_[1]
    train_labels = train_set[0]
    train_array = train_set[1]
    
    
    scaler = StandardScaler()
    train_array = scaler.fit_transform(train_array)
    test_array = scaler.transform(test_array)

    
    clf_svm.fit(train_array, train_labels)
    cv_results.append(clf_svm.score(test_array, test_labels))
    
    
print(cv_results, np.mean(cv_results), np.std(cv_results))

[0.9720149253731343, 0.8799751243781094, 0.8675373134328358, 0.976545842217484, 0.9950248756218906] 0.9382196162046907 0.05334092007623909


In [13]:
joblib.dump(clf_svm, '/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/svm_42.joblib')
joblib.dump(scaler, '/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/scaler_for_svm_42.joblib')

['/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/scaler_for_svm_42.joblib']

In [41]:
from xgboost import XGBClassifier
cv_results = []
for set_ in cv_data_wo_rep_26:
    
    
    xg_clf = XGBClassifier(n_estimators=100)
    
    
    test_set = set_[0]
    test_labels = test_set[0]
    test_array = test_set[1]
    
    
    train_set = set_[1]
    train_labels = train_set[0]
    train_array = train_set[1]
    
    xg_clf.fit(train_array, train_labels)
    cv_results.append(xg_clf.score(test_array, test_labels))
    
    
print(cv_results, np.mean(cv_results), np.std(cv_results))

[18:04:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:04:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ilya/anaconda3/envs/modelling/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ilya/anaconda3/envs/modelling/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ilya/anaconda3/envs/modelling/lib/python3.8/

[18:04:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:04:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:04:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9950248756218906, 0.9825870646766169, 0.75, 0.9992892679459844, 0.9466950959488273] 0.9347192608386639 0.09419342396139899


/home/ilya/anaconda3/envs/modelling/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [46]:
joblib.dump(xg_clf, '/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/xg_42.joblib') 

['/home/ilya/work/Projects/Article/Case_studies/script_for_model_training/models/xg_42.joblib']